
## Fleet dashboard data pipeline

In this notebook we display a compendium of functionalities through which necessary for the visualization tables are created from a set of 3 raw data sources.
The structure is straightforward to follow, there are a set of auxiliary functionalities in the beginning leading which the data is read from an Azure SQL DB, it is processed leveraging the previous functionalities and later dumped into a blob storage container as csv's

Throughout the notebook, references to what each and every table created is used for. Nonehteless, for a more exhaustive overview, please refer to the Fleet MvP documentation in which everything is in a full breath of detail 

In [0]:
##%sh pip install azure-storage-blob

In [0]:
dbutils.widgets.text("jdbcHostname", "")
jdbcHostname = dbutils.secrets.get(scope = "mdugs-scope", key = "mdugs-sqlserver-hostname") or dbutils.widgets.get("jdbcHostname")

dbutils.widgets.text("jdbcDatabase", "")
jdbcDatabase = dbutils.secrets.get(scope = "mdugs-scope", key = "mdugs-sqlserver-database-datahub") or dbutils.widgets.get("jdbcDatabase")

dbutils.widgets.text("jdbcPort", "")
jdbcPort = dbutils.secrets.get(scope = "mdugs-scope", key = "mdugs-sqlserver-port") or dbutils.widgets.get("jdbcPort")

dbutils.widgets.text("jdbcUser", "")
jdbcUser = dbutils.secrets.get(scope = "mdugs-scope", key = "mdugs-sqlserver-user") or dbutils.widgets.get("jdbcUser")

dbutils.widgets.text("jdbcPassword", "")
jdbcPassword = dbutils.secrets.get(scope = "mdugs-scope", key = "mdugs-sqlserver-pass") or dbutils.widgets.get("jdbcPassword")

properties = { "user": jdbcUser, "password": jdbcPassword, "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"}

url = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)

In [0]:
storageName = dbutils.secrets.get(scope = "mdugs-scope", key = "mdugs-storage-name")
storageKey = dbutils.secrets.get(scope = "mdugs-scope", key = "mdugs-storage-config2") 

In [0]:
#%sh pip install azure-storage-blob

In [0]:
import pandas as pd
## Added 
from azure.storage.blob import ContainerClient
##
import pandas as pd
from pandas.tseries.offsets import DateOffset
from io import StringIO
import io
import numpy as np
from pyspark.sql import *
import os
import time
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [0]:
def save_in_blob(df, file_name):
  connectStr = "DefaultEndpointsProtocol=https;AccountName={0};AccountKey={1};EndpointSuffix=core.windows.net".format(storageName,storageKey)
  containerName = "scfrota"
  containerClient = ContainerClient.from_connection_string(conn_str=connectStr, container_name=containerName)

  output = df.to_csv(index = False, encoding = "utf-8", sep = ';', decimal = ',')
  output_blob_name = file_name + ".csv"

  try:
    containerClient.upload_blob(name = output_blob_name, data = output)  
  except:  
    # if file already exists
    containerClient.delete_blob(blob=output_blob_name)
    containerClient.upload_blob(name = output_blob_name, data = output)    
  return "SAVED!"


def read_from_blob(file_name): 
  connectStr = "DefaultEndpointsProtocol=https;AccountName={0};AccountKey={1};EndpointSuffix=core.windows.net".format(storageName,storageKey)
  containerName = "scfrota"
  containerClient = ContainerClient.from_connection_string(conn_str=connectStr, container_name=containerName)
  
  try:  
    file_str = containerClient.download_blob(file_name).readall().decode("utf-8")
  except:
    return "File does not exist."
  df = pd.read_csv(StringIO(file_str), sep = ';', decimal = ',', encoding = 'utf-8')
  return(df)

In [0]:
def add_trimester(x):
  ''' Given a datetime object, returns the term in the desirable format for the dashboard display '''
  if (x.month <= 3): return '1T'
  if (x.month <= 6): return '2T'
  if (x.month <= 9): return '3T'
  return '4T'

def moment_of_day(x):
  if (x < '06'): return '4. Noite'
  if (x > '19'): return '4. Noite'
  if (x < '11'): return '1. Manhã'
  if (x < '15'): return '2. Meio dia'
  return '3. Tarde'

def bins_kms(x):
  if (x < 50000): return '1. <50k'
  if (x < 100000): return '2. 50-100k'
  if (x < 150000): return '3. 100-150k'
  if (x < 200000): return '4. 150-200k'
  if (x < 250000): return '5. 200-250k'
  if (x < 300000): return '6. 250-300k'
  if (x >= 300000): return '7. >300k'
  return '1. <50k'

def bins_years(x):
  if (x <= 2): return '1. <2 yr'
  if (x <= 4): return '2. 2-4 yr'
  if (x <= 6): return '3. 4-6 yr'
  if (x <= 8): return '4. 6-8 yr'
  if (x <= 10): return '5. 8-10 yr'
  if (x > 10): return '6. >10yr'
  return '1. <2 yr'

def days_bins(x):
  if (x < 30): return '01. <30 dias'
  if (x < 60): return '02. 30-60 dias'
  if (x < 90): return '03. 60-90 dias'
  if (x < 120): return '04. 90-120 dias'
  if (x < 150): return '05. 120-150 dias'
  if (x < 180): return '06. 150-180 dias'
  if (x < 210): return '07. 180-210 dias'
  if (x < 240): return '08. 210-240 dias'
  if (x < 270): return '09. 240-270 dias'
  if (x < 300): return '10. 270-300 dias'
  if (x < 330): return '11. 300-330 dias'
  if (x < 360): return '12. 330-360 dias'
  if (x >= 360): return '13. >360 dias'
  return '13. >360 dias'

def bins_kms2(x):
  if (x < 25000): return '01. <25k'
  if (x < 50000): return '02. 25-50k'
  if (x < 75000): return '03. 50-75k'
  if (x < 100000): return '04. 75-100k'
  if (x < 125000): return '05. 100-125k'
  if (x < 150000): return '06. 125-150k'
  if (x < 175000): return '07. 150-175k'
  if (x < 200000): return '08. 175-200k'
  if (x < 225000): return '09. 200-225k'
  if (x < 250000): return '10. 225-250k'
  if (x >= 250000): return '11. >250k'
  return '01. <25k'

def bins_years2(x):
  if (x <= 2): return '01. <2 yr'
  if (x <= 4): return '02. 2-4 yr'
  if (x <= 6): return '03. 4-6 yr'
  if (x <= 8): return '04. 6-8 yr'
  if (x <= 10): return '05. 8-10 yr'
  if (x <= 12): return '06. 10-12 yr'
  if (x <= 14): return '07. 12-14 yr'
  if (x > 14): return '08. >14 yr'
  return '01. <1 yr'

def bins_repairs2(x):
  if (x <= 20): return '1. <20'
  if (x <= 40): return '2. 20-40'
  if (x <= 60): return '3. 40-60'
  if (x <= 80): return '4. 60-80'
  if (x <= 100): return '5. 80-100'
  if (x <= 120): return '6. 100-120'
  if (x <= 140): return '7. 120-1400'
  if (x > 140): return '8. >140'
  return '1. <20'

def bins_kms3(x):
  if (x < 1000): return '1. <1k'
  if (x < 2500): return '2. 1-2.5k'
  if (x < 5000): return '3. 2.5-5k'
  if (x < 7500): return '4. 5-7.5k'
  if (x < 10000): return '5. 7.5-10k'
  if (x < 15000): return '6. 10-15k'
  if (x < 20000): return '7. 15-20k'
  if (x >= 20000): return '8. >20k'
  return '4. 5-7.5k'
  
def bins_repairs3(x):
  if (x == 0): return '0'
  if (x == 1): return '1'
  if (x  == 2): return '2'
  if (x == 3): return '3'
  if (x == 4): return '4'
  if (x == 5): return '5'
  if (x == 6): return '6'
  if (x >6): return 'Mais de 6'
  return '0'


def clean_movimentos(data):
    # Fill null values
    NAtoUnknownMask = ['mov_movement_type','mov_plate','mov_category_of_vehicle','mov_movement_description',
                       'mov_owner_company','mov_fleet_manager','mov_status','mov_odometer_unit','mov_supplier','mov_operation',
                       'mov_sub_operation','mov_exit','mov_currency', 'mov_accident_location', 'mov_cost_center']
    
    data.loc[:, NAtoUnknownMask] = data.loc[:, NAtoUnknownMask].fillna("UNKNOWN")
    
    # Specify the country for a better location in the dashboard maps
    data['mov_accident_location'] = data['mov_accident_location'].astype(str)+', Portugal'
    data['mov_exit'] = data['mov_exit'].astype(str)+', Portugal'
    
    # Clean the owner company variable 
    dict_owner_company = dict(zip(data.mov_owner_company.unique(), data.mov_owner_company.unique()))
    dict_owner_company['EDP DISTRIBUIÇAO'] = 'EDP DISTRIBUIÇÃO'
    dict_owner_company['EDP GESTAO DA PRODUÇAO DE ENERGIA'] = 'EDP GESTÃO DA PRODUÇÃO DE ENERGIA'
    dict_owner_company['EDP SOLUÇOES COMERCIAIS'] = 'EDP SOLUÇÕES COMERCIAIS'
    dict_owner_company['FUNDAÇAO EDP'] = 'FUNDAÇÃO EDP'
    dict_owner_company['"EDP INTERNACIONAL, SA"'] = 'EDP INTERNACIONAL, SA'
    dict_owner_company['EDP INOVAÇAO'] = 'EDP INOVAÇÃO'
    dict_owner_company['SAVIDA'] = 'SÃVIDA'
    data['mov_owner_company'] = data['mov_owner_company'].map(dict_owner_company)

    # Consider as accident every movement with a Accident ID associated
    data['mov_accident'] = 'Yes'
    data.loc[data['mov_accident_id'].isnull(), 'mov_accident'] = 'No'
    data.loc[data['mov_accident_id'] == '', 'mov_accident'] = 'No'

    # Scheduled: changed to Yes/No
    # Operations related to accidents appear as scheduled, marking them
    # as no scheduled because the cause was an accident
    scheduledMask = (~data['mov_scheduled'].isnull()) & (data['mov_scheduled']!='') & (data['mov_accident'] == 'No')
    data.loc[~scheduledMask, 'mov_scheduled'] = 'No'
    data.loc[scheduledMask, 'mov_scheduled'] = 'Yes'

    # Creating new feature to split movements by type
    data["mov_movement_category"] = "DIRECT_COST_USAGE"
    maskType = ((data["mov_movement_type"] == "SERVIÇOS") | 
                (data["mov_movement_type"] == "REQUISIÇÃO OFICINA") | 
                (data["mov_movement_type"] == "TIRES") | 
                (data["mov_movement_type"] == "IPO")) & (data["mov_accident"] == "No")

    data.loc[maskType, "mov_movement_category"] = "MAINTENANCE"
    data.loc[maskType, "mov_movement_type"] = "NO SCHEDULED"
    maskScheduled = data["mov_scheduled"] == "Yes"
    data.loc[maskScheduled, "mov_movement_type"] = "SCHEDULED"
    maskSinistro = data["mov_accident"] == "Yes"
    data.loc[maskSinistro, "mov_movement_type"] = "SINISTRO"
    maskTires = ((data["mov_sub_operation"] == "ECVAL") | (data["mov_sub_operation"] == "ECVAL") | 
                 (data["mov_sub_operation"] == "PNE01") | (data["mov_sub_operation"] == "PNE02") | 
                 (data["mov_sub_operation"] == "PNE02") | (data["mov_sub_operation"] == "PNE98"))
    data.loc[maskTires, "mov_movement_type"] = "TIRES"
    data.drop("mov_accident_id", axis = 1, inplace = True)
    
    # Treat movement date as a date
    data.mov_movement_date = pd.to_datetime(data.mov_movement_date)
    data = data[data.mov_movement_date >= pd.Timestamp('2018-01-01')]
    
    return data



def clean_viaturas(data):
    # Fill null values
    data.loc[:, data.columns[data.dtypes == "object"]] = data.loc[:, data.columns[data.dtypes == "object"]].fillna("UNKNOWN")
    data.loc[:, data.columns[data.dtypes == "int64"]] = data.loc[:, data.columns[data.dtypes == "int64"]].fillna(-1)
    data.loc[:, data.columns[data.dtypes == "float64"]] = data.loc[:, data.columns[data.dtypes == "float64"]].fillna(-1)
    
    # Add owner company based on code of empresa proprietaria
    mapping_owner_company = {'G1000': 'EDP ENERGIAS DE PORTUGAL', 'G2000': 'EDP GESTÃO DA PRODUÇÃO DE ENERGIA','G2130': 'TERGEN', 'G2610': 'EDP GESTÃO DA PRODUÇÃO DE ENERGIA',
                             'G2800': 'EDP DISTRIBUIÇÃO','G2820': 'EDP SERVIÇO UNIVERSAL','G2822': 'EDP DISTRIBUIÇÃO','G2900': 'EDP COMERCIAL','G3255': 'EDPR PT - PROMOÇÃO E OPERAÇÃO S A',
                             'G4000': 'REN GÁS DISTRIBUIÇAO SGPS, S.A.','G4170': 'REN PORTGÁS DISTRIBUIÇAO, S.A.','G4250': 'EDP VALOR GEST.INT.SERV.','G4260': 'EDP GÁS SERVIÇO UNIVERSAL',
                             'G6310': 'EDP IMOBILIÁRIA E PARTIC.','G6330': 'LABELEC','G6340': 'SÃVIDA','G6370': 'EDP INTERNACIONAL, SA','G6390': 'EDP VALOR GEST.INT.SERV.',
                             'G6890': 'EDP SOLUÇÕES COMERCIAIS','G6900': 'EDP ESTUDOS E CONSULTORIA','G6940': 'EDP ENERGIAS DE PORTUGAL','G6990': 'FUNDAÇÃO EDP','G6150': 'Home Energy II',
                             'G2110': 'OEM SERVIÇOS'}
    data['via_company'] = data.via_emp_prop.map(mapping_owner_company)
    
    # Homogenize the fuel variable
    mapping_fuel = {'GASÓLEO': 'Combustão', 'ENERGIA ELÉCTRICA': 'Elétricos e Plug-Ins', 'PLUG-IN GASOLINA': 'Elétricos e Plug-Ins',
                    'GASOLINA': 'Combustão', 'SEM COMBUSTÍVEL': 'Sem Combustivel', 
                    'HÍBRIDO GASOLINA': 'Híbridos', 'GASOLINA/GAS NATURAL': 'Combustão', 'HÍBRIDO GASÓLEO': 'Híbridos', 
                    'GÁS PROPANO GARRAFA': 'Combustão', 'PLUG-IN GASÓLEO': 'Elétricos e Plug-Ins'}
    data['via_fuel'] = data['via_fuel'].map(mapping_fuel)
    
    # Homogenize the attribution variable
    mapping_attribution = {'SERVIÇO':  'Viaturas de Serviço', 'VUP': 'VUPs', 'SERVIÇO PIQUETE': 'Viaturas de Serviço', 'POOL GERAL':  'POOL GERAL', 'VUP/CA': 'VUPs',
                           'POOL VUP/VS': 'POOL VUP/VS','POOL CA': 'POOL CA'}
    data['via_attribution'] = data['via_attribution'].map(mapping_attribution)
    
    # Add snapshot variables
    data['snapshot_date'] = pd.to_datetime(data['via_year'].astype(str)+'-'+data['via_month'].astype(str)) + pd.offsets.MonthEnd(0)
    data['trimester'] = data['snapshot_date'].apply(lambda x: add_trimester(x))
    data['is_last_snapshot'] = data['snapshot_date'] == data['snapshot_date'].max()
    
    # Add viatura age based on today's date
    data.via_date_of_registration = pd.to_datetime(data.via_date_of_registration)
    data.via_modified = pd.to_datetime(data.via_modified)
    data['years_old'] = (pd.Timestamp.today() - data.via_date_of_registration).apply(lambda x: (x.days/365))

    return data

In [0]:
def load_viaturas():
  print('Loading viaturas Master table :::', time.ctime())
  # READ DATA FROM SQL DB
  pushdown_query_viatura = "(select * from [dbo].[dim_viatura]) emp_id"
  viaturas = spark.read.jdbc(url=url, table=pushdown_query_viatura, properties=properties).toPandas()
  
  # RENAME 
  config_viaturas = read_from_blob('/in/config/0_config_viaturas.csv')
  viaturas = viaturas[config_viaturas.column]
  viaturas = viaturas.astype(dict(zip(config_viaturas.column, config_viaturas.newtype)))
  viaturas.columns = config_viaturas.new_name
  
  # CLEAN 
  viaturas = clean_viaturas(viaturas)
  return viaturas


def load_movimentos():
  print('Loading movimentos Master table :::', time.ctime())
  # READ DATA FROM SQL DB
  pushdown_query_movimentos = "(select * from [dbo].[fact_movimentos]) emp_id"
  movimentos = spark.read.jdbc(url=url, table=pushdown_query_movimentos, properties=properties).toPandas()
  
  # RENAME
  config_movimentos = read_from_blob('/in/config/0_config_movimentos.csv')
  movimentos = movimentos.astype(dict(zip(config_movimentos.column, config_movimentos.newtype)))
  movimentos.columns = config_movimentos.new_name
  
  # CLEAN
  movimentos_clean = clean_movimentos(movimentos)
  return movimentos_clean

def load_cadastro():
  print('Loading Cadastro Master table :::', time.ctime())
  # READ DATA FROM SQL DB
  pushdown_query_rh = "(select * from [dbo].[dim_rh]) emp_id"
  rh = spark.read.jdbc(url=url, table=pushdown_query_rh, properties=properties).toPandas()
  
  # RENAME
  config_cadastro = read_from_blob('/in/config/0_config_cadastro.csv')
  rh = rh[config_cadastro.column]
  rh = rh.astype(dict(zip(config_cadastro.column, config_cadastro.newtype)))
  rh.columns = config_cadastro.new_name

  return rh

In [0]:
def filters_table(clean_viaturas):
  '''Auxiliary function to create the table used for the right hand side slicers in PowerBI
  Returns a table with licence plate as a unique key and all the observed combinations of the desired filters: status, category, fuel,
  attribution, cost center, company, classification, city and brand'''
  
  filters = clean_viaturas.sort_values('snapshot_date', ascending = False).drop_duplicates('via_plate')[['via_plate','via_status', 'via_category', 
                                                                                                         'via_fuel', 'via_attribution', 'via_cost_center', 
                                                                                                         'via_company', 'via_classification', 'via_city', 
                                                                                                         'via_brand']].reset_index(drop = True)
  return filters

In [0]:
def fleet_status_table(clean_movimentos, clean_viaturas, cadastro):
    print('1. Creating Fleet-Status table :::', time.ctime())
    
    # Select only operative vehicles
    fleet_status = clean_viaturas[clean_viaturas.via_status.isin(['IMOB. POR ACIDENTE','OPERACIONAL'])]

    # Add age and KM bucket
    fleet_status['km_bucket'] = fleet_status.via_total_kms.apply(lambda x: bins_kms(x))
    fleet_status['age_bucket'] = (fleet_status.years_old).apply(lambda x: bins_years(x))
    
    # Add electric variables
    fleet_status.loc[fleet_status.via_fuel == 'ENERGIA ELÉCTRICA', 'electric_km'] = fleet_status['via_total_kms']
    fleet_status.loc[fleet_status.via_fuel != 'ENERGIA ELÉCTRICA', 'non_electric_km'] = fleet_status['via_total_kms']
    fleet_status['electric_km'].fillna(0, inplace = True)
    fleet_status['non_electric_km'].fillna(0, inplace = True)
    fleet_status['is_electric'] = (fleet_status.via_fuel == 'Elétricos e Plug-Ins').astype(float)*100

    # Add variables related to KM based on abastecimentos from movimentos
    abastecimentos = clean_movimentos[clean_movimentos.mov_movement_type == 'ABASTECIMENTO']
    fleet_status['number_abastecimentos'] = fleet_status.via_plate.map(abastecimentos.groupby('mov_plate').size()).fillna(0)
    fleet_status['number_sch_maintenance'] = fleet_status.via_plate.map(clean_movimentos[clean_movimentos.mov_movement_type == 'SCHEDULED'].groupby('mov_plate').size()).fillna(0)
    fleet_status['number_nosch_maintenance'] = fleet_status.via_plate.map(clean_movimentos[clean_movimentos.mov_movement_type == 'NO SCHEDULED'].groupby('mov_plate').size()).fillna(0)
    fleet_status['number_tires'] = fleet_status.via_plate.map(clean_movimentos[clean_movimentos.mov_movement_type == 'TIRES'].groupby('mov_plate').size()).fillna(0)
    fleet_status['number_tolls'] = fleet_status.via_plate.map(clean_movimentos[clean_movimentos.mov_movement_type == 'PORTAGEMS'].groupby('mov_plate').size()).fillna(0)

    fleet_status['first_abastecimento'] = fleet_status.via_plate.map(abastecimentos.groupby('mov_plate').mov_movement_date.min()).fillna(pd.Timestamp('1900-01-01'))
    fleet_status['last_abastecimento'] = fleet_status.via_plate.map(abastecimentos.groupby('mov_plate').mov_movement_date.max()).fillna(pd.Timestamp('1900-01-01'))
    fleet_status['days_since_first_abastecimento'] = (fleet_status['last_abastecimento'] - fleet_status['first_abastecimento']).apply(lambda x: x.days).fillna(0)+1
    fleet_status['liters_consumed'] = fleet_status.via_plate.map(abastecimentos.groupby('mov_plate').mov_quantity.sum()).fillna(0)
    
    
    # Add number of collaborators by viatura variable and increase / decrease indicators
    fleet_status['colabviatura'] = round(fleet_status[fleet_status.is_last_snapshot == True].shape[0] / 
                                          cadastro[(cadastro.cad_headcount == 'Sim') & (cadastro.cad_year == 2019) & (cadastro.cad_month == 9)].cad_colaborador_num.nunique(),2)
    fleet_status['increasecolab'] = -0.01
    
    cars_one_year_before = fleet_status[fleet_status.snapshot_date == sorted(fleet_status.snapshot_date.unique())[-12]].via_plate.nunique()
    fleet_status['increaseviaturas'] = round((fleet_status['is_last_snapshot'].sum()-cars_one_year_before) / cars_one_year_before,2)
    
    
    # Select columns to stick with
    fleet_status_cols = ['snapshot_date', 'is_last_snapshot', 'via_plate', 'km_bucket', 'age_bucket', 'via_total_kms',
                         'electric_km', 'non_electric_km', 'via_co2_emissions', 'via_brand', 'via_city',
                         'via_category', 'via_fuel', 'via_attribution', 'via_company', 'via_acquisition_value',
                         'years_old', 'number_abastecimentos', 'first_abastecimento', 'last_abastecimento',
                         'days_since_first_abastecimento', 'liters_consumed', 'is_electric',  'via_status', 
                         'number_sch_maintenance', 'number_nosch_maintenance', 'number_tires', 'number_tolls', 'via_cost_center', 
                         'trimester', 'colabviatura', 'increaseviaturas', 'increasecolab']
    
    fleet_status = fleet_status[fleet_status_cols]
    
    return fleet_status

In [0]:
def fleet_balances_table(clean_viaturas, clean_movimentos):
  '''Auxiliary function that returns a table with all the viaturas per snapshot indicating if they were an transfer (entering or exiting) and furhtermore, 
  denoting whether it was internal or not'''

  print('2. Creating Fleet-Balances table :::', time.ctime())

  fleet_balances = clean_viaturas[['snapshot_date', 'via_plate', 'via_status','via_attribution',
                                  'via_brand', 'via_city', 'via_company', 'via_category', 'via_fuel',
                                  'via_date_of_registration', 'via_cost_center', 'trimester']].sort_values('snapshot_date')

  fleet_balances['snapshot_plate'] = clean_viaturas['snapshot_date'].astype(str)+'_'+clean_viaturas['via_plate'].astype(str)
  fleet_balances['is_last_period'] = (fleet_balances['snapshot_date'] >= pd.to_datetime(str(fleet_balances['snapshot_date'].max().year)+'-01')).astype(float)
  fleet_balances['is_active'] = fleet_balances['snapshot_plate'].map(
      fleet_balances.groupby('snapshot_plate').via_status.apply(lambda x: ('OPERACIONAL' in list(x)) or ('IMOB. POR ACIDENTE' in list(x))))

  # Inclusion of the previous_active flag indicating whether a plate in a given snapshot was active in the past month
  previous_state = dict()
  for i in fleet_balances.snapshot_date.unique():
      fleet_balances.loc[fleet_balances.snapshot_date == i, 'previous_active'] = fleet_balances.via_plate.map(previous_state)
      previous_state = fleet_balances.loc[fleet_balances.snapshot_date == i].groupby('via_plate').is_active.max().fillna(0)
  fleet_balances['previous_active'].fillna(False, inplace = True)

  # Inclusion of flags to indicate if the combination of plate and snapshot changed to inactive or to active
  fleet_balances['changed_inactive'] = ((fleet_balances['is_active'] == False) & (fleet_balances.previous_active == True)).astype(float)
  fleet_balances.loc[fleet_balances.changed_inactive == 1,'changed_inactive'] = -1
  fleet_balances['changed_active'] = ((fleet_balances['is_active'] == True) & (fleet_balances.previous_active == False)).astype(float)
  fleet_balances.loc[fleet_balances.snapshot_date <= '2018-02-01', 'changed_active'] = 0
  fleet_balances['abs_changed_inactive'] = abs(fleet_balances.changed_inactive)

  # Add internal sale flag indicating whether the cange was made to the exterior or it was an internal movement
  # We leverage the last date of observation for each car to determine the last selling date
  fleet_balances['last_active_date'] = fleet_balances.via_plate.map(fleet_balances[fleet_balances.is_active].groupby('via_plate')['snapshot_date'].max())
  fleet_balances['first_active_date'] = fleet_balances.via_plate.map(fleet_balances[fleet_balances.is_active].groupby('via_plate')['snapshot_date'].min())
  fleet_balances['internal_sale'] = 0
  fleet_balances.loc[(fleet_balances.changed_inactive != 0) & (fleet_balances.snapshot_date < fleet_balances.last_active_date ), 'internal_sale'] = 1
  fleet_balances.loc[(fleet_balances.changed_active != 0) & (fleet_balances.snapshot_date > fleet_balances.first_active_date ), 'internal_sale'] = 1
  fleet_balances = fleet_balances.drop_duplicates('snapshot_plate', keep = 'first')


  # Add purchase details
  fleet_balances['via_acquisition_value'] = -fleet_balances.via_plate.map(clean_viaturas.groupby('via_plate').via_acquisition_value.max())
  fleet_balances['abs_via_acquisition_value'] = abs(fleet_balances['via_acquisition_value'])
  fleet_balances['via_actual_selling_price'] = fleet_balances.via_plate.map(clean_viaturas.groupby('via_plate').via_actual_selling_price.max())

  # Add KM and age bucket variables
  fleet_balances['km'] = fleet_balances.via_plate.map(clean_viaturas.groupby('via_plate').via_total_kms.max())
  fleet_balances['km_bucket'] = fleet_balances.km.apply(lambda x: bins_kms(x))

  # Overwrite years old variable baesd on last active date instead of today's date
  fleet_balances.loc[fleet_balances.last_active_date.notnull(),'years_old'] = (fleet_balances.last_active_date - fleet_balances.via_date_of_registration).dt.days / 365
  fleet_balances['age_bucket'] = fleet_balances.years_old.apply(lambda x: bins_years(x))


  # Select columns to stick with
  fleet_balance_cols = ['snapshot_date','via_plate','via_status','via_attribution','via_brand','via_city','via_company','via_category','via_fuel','via_date_of_registration',
                        'via_cost_center','trimester','snapshot_plate','is_last_period', 'is_active', 'previous_active','changed_inactive','changed_active',
                        'abs_changed_inactive','last_active_date',	'first_active_date',	'internal_sale',	'via_acquisition_value',	'abs_via_acquisition_value',
                        'via_actual_selling_price',	'km',	'km_bucket','years_old', 'age_bucket']
  fleet_balances = fleet_balances[fleet_balance_cols]

  # Rename certain fields for a portuguese visualization in PowerBI
  fleet_balances.rename(columns = {'changed_inactive': 'Mudança pana não activo', 'changed_active': 'Mudança pana activo'}, inplace = True)

  return fleet_balances

In [0]:
def capex_table(fleet_balances, clean_viaturas, clean_movimentos):
  '''Auxiliary function that creates a table with the capital transfers due to entries and exits from the fleet. 
  It heavily relies on the fleet balances computed previously. Two types of transfers are concatenated in the same format to create the final capex table, acquisitions and dismissals'''

  print('3. Creating CAPEX table :::', time.ctime())

  # Define acquisitions
  acquisitions = fleet_balances[(fleet_balances['internal_sale'] == 0) & (fleet_balances['Mudança pana activo'] == 1)][['via_plate', 'snapshot_date']]
  acquisitions.columns = ['via_plate', 'last_date']
  acquisitions['first_date'] = acquisitions.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_date_of_registration)))
  acquisitions['via_category'] = acquisitions.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_category)))
  acquisitions['via_fuel'] = acquisitions.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_fuel)))
  acquisitions['years_old'] = acquisitions.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.years_old)))
  acquisitions['via_attribution'] = acquisitions.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_attribution)))
  acquisitions['via_acquisition_value'] = -acquisitions.via_plate.map(clean_viaturas.groupby('via_plate').via_acquisition_value.max())
  acquisitions['via_actual_selling_price'] = 0
  acquisitions['km'] = acquisitions.via_plate.map(clean_viaturas.groupby('via_plate').via_total_kms.max())
  acquisitions['below_250k'] = (acquisitions['km'] < 250000).astype(float)
  acquisitions['below_8yrs'] = (acquisitions['years_old'] < 8).astype(float)
  acquisitions['acquired'] = 1
  acquisitions['sold'] = 0
  acquisitions['status'] = 'BOUGHT'

  # Define dismissals
  dismissals = fleet_balances[(fleet_balances['internal_sale'] == 0) & (fleet_balances['Mudança pana não activo'] == -1)][['via_plate', 'snapshot_date']]
  dismissals.columns = ['via_plate', 'last_date']
  selling_statuses = ['ENTREGUE À LEILOEIRA', 'IDENTIFICADA PARA DO', 'IMOB. PARA VENDA', 'VENDIDA']
  min_dates = clean_viaturas[clean_viaturas.via_status.isin(selling_statuses)].groupby('via_plate')['via_modified'].min()
  max_dates = clean_viaturas[clean_viaturas.via_status.isin(selling_statuses)].groupby('via_plate')['via_modified'].max()
  dismissals['first_date'] = dismissals.via_plate.map(min_dates)
  dismissals['via_category'] = dismissals.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_category)))
  dismissals['via_fuel'] = dismissals.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_fuel)))
  dismissals['years_old'] = (dismissals['last_date'] - dismissals.via_plate.map(dict(zip(
      clean_viaturas.via_plate,clean_viaturas.via_date_of_registration)))).apply(lambda x: (x.days/365))
  dismissals['via_attribution'] = dismissals.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_attribution)))
  dismissals['via_acquisition_value'] = 0
  dismissals['via_actual_selling_price'] = dismissals.via_plate.map(clean_viaturas.groupby('via_plate').via_actual_selling_price.max())
  dismissals['km'] = dismissals.via_plate.map(clean_viaturas.groupby('via_plate').via_total_kms.max())
  dismissals['below_250k'] = (dismissals['km'] < 250000).astype(float)
  dismissals['below_8yrs'] = (dismissals['years_old'] < 8).astype(float)
  dismissals['acquired'] = 0
  dismissals['sold'] = 1
  dismissals['status'] = 'SOLD'

  # Concatenate dismissals and acquisitions to get the full capex table
  capex = pd.concat([acquisitions, dismissals]).reset_index(drop = True)
  capex['abs_acquisition_value'] = abs(capex['via_acquisition_value'])
  capex['cost_center'] = capex.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_cost_center))).astype(str)
  capex['company'] = capex.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_company))).astype(str)
  capex['is_last_period'] = (capex.last_date >= pd.to_datetime(str(capex.last_date.max().year)+'-01')).astype(float)
  capex['trimester'] = capex['last_date'].apply(lambda x: add_trimester(x))


  # Add age and km buckets for the visualizations
  capex['km_bucket'] = capex.km.apply(lambda x: bins_kms(x))
  capex['age_bucket'] = capex.years_old.apply(lambda x: bins_years(x))

  # Select columns to stick with
  capex_cols = ['via_plate',	'last_date',	'first_date',	'via_category',	'via_fuel',	'years_old',	'via_attribution',	'via_acquisition_value',	
                'via_actual_selling_price',	'km',	'below_250k',	'below_8yrs',	'acquired',	'sold',	'status',	'abs_acquisition_value',	
                'cost_center','company','is_last_period',	'trimester',	'km_bucket',	'age_bucket']
  capex = capex[capex_cols]

  # Rename certain fields for the adequate portuguese visualization
  capex.rename(columns = {'via_acquisition_value': 'Valor de aquisição', 'via_actual_selling_price': 'Valor de venda'}, inplace = True)

  return capex

In [0]:
def opex_table(clean_viaturas, clean_movimentos):
    ''' Auxiliary function that returns the opex table. A table with the clean movements and additional indicators that is ready to be visualized in the dashboard '''
  
    print('4. Creating OPEX table :::', time.ctime())
    
    opex_breakdown = clean_movimentos[['mov_plate', 'mov_movement_date','mov_movement_time','mov_movement_type', 'mov_kms', 
                                       'mov_quantity', 'mov_total_net_price', 'mov_supplier', 'mov_owner_company',
                                       'mov_movement_description', 'mov_accident_location', 'mov_exit','mov_cost_center', 
                                       'mov_category_of_vehicle', 'mov_status']]
    
    opex_breakdown['trimester'] = opex_breakdown['mov_movement_date'].apply(lambda x: add_trimester(x))
    opex_breakdown['is_last_period'] = (opex_breakdown['mov_movement_date'] >= pd.to_datetime(str(opex_breakdown['mov_movement_date'].max().year)+'-01')).astype(float)
    opex_breakdown['plate_date'] = opex_breakdown['mov_plate'].astype(str)+'_'+opex_breakdown['mov_movement_date'].astype(str)
    opex_breakdown['days_since_last_movement'] = opex_breakdown.groupby('mov_plate').mov_movement_date.diff().dt.days
    opex_breakdown['days_since_last_maintenance'] = opex_breakdown[opex_breakdown.mov_movement_type == 'SCHEDULED'].groupby('mov_plate').mov_movement_date.diff().dt.days
    opex_breakdown['days_since_last_tires'] = opex_breakdown[opex_breakdown.mov_movement_type == 'TIRES'].groupby('mov_plate').mov_movement_date.diff().dt.days
    opex_breakdown.days_since_last_movement = opex_breakdown.days_since_last_movement.fillna(-1)
    opex_breakdown.days_since_last_maintenance = opex_breakdown.days_since_last_maintenance.fillna(-1)
    opex_breakdown.days_since_last_tires = opex_breakdown.days_since_last_tires.fillna(-1)

    # Add buckets for maintenance and tires
    opex_breakdown['bucket_last_maintenance'] = opex_breakdown['days_since_last_maintenance'].apply(lambda x: days_bins(x))
    opex_breakdown['bucket_last_tires'] = opex_breakdown['days_since_last_tires'].apply(lambda x: days_bins(x))
    
    
    # Append static movement information
    opex_breakdown['mov_movement_description']= opex_breakdown.mov_movement_description.apply(lambda x: str(x).replace('“','').replace('”','').replace('–',''))
    opex_breakdown['via_brand'] = opex_breakdown['mov_plate'].map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_brand)))
    opex_breakdown['via_classification'] = opex_breakdown['mov_plate'].map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_classification)))
    opex_breakdown['via_category'] = opex_breakdown['mov_plate'].map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_category)))
    opex_breakdown['via_fuel'] = opex_breakdown['mov_plate'].map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_fuel))).fillna('GASÓLEO')
    opex_breakdown['via_attribution'] = opex_breakdown['mov_plate'].map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_attribution)))
    opex_breakdown['via_city'] = opex_breakdown['mov_plate'].map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_city)))
    opex_breakdown['eur_liter'] = (opex_breakdown['mov_total_net_price'] / opex_breakdown['mov_quantity']).fillna(0)
    
    # Collapse certain movement types to have a reduced movement type
    opex_breakdown.loc[opex_breakdown.mov_movement_type == 'MULTA', 'mov_movement_type'] = 'OUTROS CUSTOS'
    reduced_type_dict = dict(zip(opex_breakdown.mov_movement_type.unique(),opex_breakdown.mov_movement_type.unique()))
    reduced_type_dict['NO SCHEDULED'] = 'MAINTENANCE'
    reduced_type_dict['SCHEDULED'] =  'MAINTENANCE'
    reduced_type_dict['TIRES'] = 'MAINTENANCE'
    opex_breakdown['reduced_mov_type'] = opex_breakdown.mov_movement_type.map(reduced_type_dict)
    
    # Add moment of day variables
    opex_breakdown['moment_of_day'] = opex_breakdown.mov_movement_time.apply(lambda x: moment_of_day(str(x)[:2]))
    dictionary_weekday = {0: '1. Segunda-feira', 1: '2. Terça-feira', 2: '3. Quarta-feira', 3: '4. Quinta-feira', 4: '5. Sexta-feira',
                             5: '6. Sábado', 6: '7. Domingo'}
    opex_breakdown.mov_movement_date = pd.to_datetime(opex_breakdown.mov_movement_date)
    opex_breakdown['weekday'] = opex_breakdown.mov_movement_date.apply(lambda x: dictionary_weekday[x.weekday()])


    # ADD DATA OF EUR / KM / QUARTER
    opex_breakdown['plate_quarter'] = (opex_breakdown['mov_plate'].astype(str)+'_'+
                                       pd.to_datetime(opex_breakdown.mov_movement_date).apply(lambda x: x.quarter).astype(str))
    opex_breakdown['km_in_quarter'] = opex_breakdown['plate_quarter'].map(
        opex_breakdown.groupby('plate_quarter').mov_kms.apply(lambda x: max(x)-min(x)))
    opex_breakdown['km_in_quarter'] = opex_breakdown['km_in_quarter'].fillna(1)
    ncars = opex_breakdown.mov_plate.nunique()
    opex_breakdown['eur_km_quarter'] = (opex_breakdown['mov_total_net_price'] / opex_breakdown['km_in_quarter']) / ncars
    
    
    # Select columns to stick with
    opex_cols = ['mov_plate', 'mov_movement_date', 'mov_movement_time','mov_movement_type','mov_kms','mov_quantity','mov_total_net_price','mov_supplier','mov_owner_company',
                 'mov_movement_description','mov_accident_location','mov_exit','mov_cost_center','mov_category_of_vehicle','mov_status','trimester','is_last_period',
                 'plate_date','days_since_last_movement','days_since_last_maintenance','days_since_last_tires','bucket_last_maintenance','bucket_last_tires',
                 'via_brand','via_classification','via_category','via_fuel','via_attribution','via_city','eur_liter','reduced_mov_type','moment_of_day','weekday',
                 'plate_quarter','km_in_quarter','eur_km_quarter']
    
    opex_breakdown = opex_breakdown[opex_cols]
    
    return opex_breakdown

In [0]:
def portagens_times_table(movimentos):
  '''This auxiliary function is a temporary solution for the lack of an adecuate time record of the movements. 
  It returns a table containing licence plate, location, entry and exit time of the toll even tas well as the best estimate for the time difference under the name diff'''
  
  # Select reduced group of variables
  portagens_time = movimentos[['mov_plate', 'mov_movement_date','mov_movement_description', 'mov_exit', 'mov_movement_time', 'mov_exit_time' ]]
  
  # Define time difference between entry and exit to the portagen in minutes
  entry = pd.to_datetime(portagens_time['mov_movement_date'].astype(str) + ' '+ portagens_time['mov_movement_time'].astype(str), errors='coerce')
  exit = pd.to_datetime(portagens_time['mov_movement_date'].astype(str) + ' '+ portagens_time['mov_exit_time'].astype(str), errors='coerce')
  portagens_time['diff'] = (exit-entry).dt.seconds / 60
  
  # Restrict the returning table to Portagens and include auxiliary for the visualization fields
  portagens = portagens_time[movimentos.mov_movement_type == 'PORTAGEM']
  portagens['mov_exit'] = portagens['mov_exit'].apply(lambda x: x.replace(', Portugal', ''))
  portagens['entry_exit'] = 'De ' + portagens['mov_movement_description'] + ' para ' + portagens['mov_exit']

  return portagens

In [0]:
def compute_costcalculator_table(clean_viaturas, clean_movimentos):
  
    '''Auxiliary function that given viaturas master table and movimentos master table returns the fatigue table. A table containing cost per KMs of various operational
    activities'''
    
    print('5. Creating cost calculator table :::', time.ctime())
    
    # Select only the last snapshot operative vehicles from Servicio, Ligeiros and Combustível and a handful of variables for the cost calculator
    fatigue = clean_viaturas[clean_viaturas['snapshot_date'] == clean_viaturas['snapshot_date'].max()]
    fatigue = fatigue[fatigue.via_status.isin(['IMOB. POR ACIDENTE','OPERACIONAL'])]
    fatigue_cols = ['via_plate', 'via_status','via_category', 'via_company','via_classification', 'via_brand', 'via_model',
                    'via_fuel','via_attribution', 'via_total_kms','via_date_of_registration', 'via_cc_voltage', 'via_gross_weight', 'years_old']
    fatigue = fatigue[fatigue_cols]
    fatigue = fatigue[(fatigue.via_attribution == 'Viaturas de Serviço') & (fatigue.via_category == 'LIGEIRO') & (fatigue.via_fuel == 'Combustão')]
    
    # Look only at one year ago worth of movements to estimate the expense by KMs driven
    clean_movimentos = clean_movimentos[clean_movimentos.mov_movement_date >= clean_movimentos.mov_movement_date.max() - pd.Timedelta('365 days')]
    
    # Assign the done KMs variable based on the corrected result of a regression of the liters consumed, the voltage and the gross weight of the vehicle
    fatigue['liters'] = fatigue.via_plate.map(clean_movimentos[clean_movimentos.mov_movement_type == 'ABASTECIMENTO'
                                                                ].groupby('mov_plate').mov_quantity.sum())
    fatigue['done_kms'] = fatigue['liters'] * (2.14111935e+01 -  2.15275095e-04 * fatigue['via_cc_voltage'] - 4.03502335e-03 * fatigue['via_gross_weight'])
    fatigue.loc[(fatigue.done_kms<=0), 'done_kms'] = fatigue.liters * 13
                                                  
    number_repairs = clean_movimentos[clean_movimentos.mov_movement_type.isin(['NO SCHEDULED','SCHEDULED','TIRES'])].groupby('mov_plate').mov_quantity.sum()
    fatigue['number_repairs'] = fatigue.via_plate.map(number_repairs)
    
    
    # Add cost per 1000Km indicators from different activities (maintenance, refuelling, insurance, tolls)
    cost_maintenance = clean_movimentos[clean_movimentos.mov_movement_type.isin(['NO SCHEDULED','SCHEDULED','TIRES'])].groupby('mov_plate').mov_total_net_price.sum()
    fatigue['cost_km_maintenance'] = fatigue.via_plate.map(cost_maintenance) / fatigue['done_kms']*1000

    cost_abastecimento = clean_movimentos[clean_movimentos.mov_movement_type=='ABASTECIMENTO'].groupby('mov_plate').mov_total_net_price.sum()
    fatigue['cost_km_abastecimento'] = fatigue.via_plate.map(cost_abastecimento) / fatigue['done_kms'] * 1000

    cost_seguros = clean_movimentos[clean_movimentos.mov_movement_type=='SEGUROS'].groupby('mov_plate').mov_total_net_price.sum()
    fatigue['cost_km_seguros'] = fatigue.via_plate.map(cost_seguros) / fatigue['done_kms']* 1000

    cost_portagem = clean_movimentos[clean_movimentos.mov_movement_type=='PORTAGEM'].groupby('mov_plate').mov_total_net_price.sum()
    fatigue['cost_km_portagem'] = fatigue.via_plate.map(cost_portagem) / fatigue['done_kms']* 1000

    total_cost = clean_movimentos.groupby('mov_plate').mov_total_net_price.sum()
    fatigue['cost_km_total'] = fatigue.via_plate.map(total_cost) / fatigue['done_kms']* 1000

    fatigue.fillna(0, inplace = True)
    fatigue.replace(np.inf, 0, inplace = True)

    # Add buckets of age, KM done and number of repairs for the visualization
    fatigue['km_bucket'] = fatigue.via_total_kms.apply(lambda x: bins_kms2(x))
    fatigue['years_bucket'] = fatigue.years_old.apply(lambda x: bins_years2(x))
    fatigue['repairs_bucket'] = fatigue.number_repairs.apply(lambda x: bins_repairs2(x))
    
    # Select columns to stick with
    fatigue_cols = ['via_plate','via_status','via_category','via_company','via_classification','via_brand','via_model','via_fuel','via_attribution',
                    'via_total_kms','via_date_of_registration','done_kms','years_old','number_repairs','cost_km_maintenance','cost_km_abastecimento','cost_km_seguros',
                    'cost_km_portagem','cost_km_total','km_bucket','years_bucket','repairs_bucket']
    fatigue = fatigue[fatigue_cols]
    
    return fatigue

In [0]:
def electrification_table(clean_viaturas):
  '''Auxiliary function to create the table that tracks the progress of the fleet electrification against the plan'''
  
  print('6. Creating electrification table :::', time.ctime())
  
  # Select operative vehicles and add electric flag to later compute the percentages
  fleet_status = clean_viaturas[clean_viaturas.via_status.isin(['IMOB. POR ACIDENTE','OPERACIONAL'])]
  fleet_status['is_electric'] = (fleet_status.via_fuel == 'Elétricos e Plug-Ins').astype(float)
  
  # Define the elect table that will contain the year, plan, actual and ratio of success in the electrification plan
  elect = pd.DataFrame({'year': [2018,2019,2020,2021,2022,2023,2024,2025, 2026, 2027, 2028,2029,2030],
                        'plan': [11,12,16,22,30,35,43,51,63,70,78, 90,100]})
  dict_electrification = dict(zip(elect.year,
                                  [fleet_status[(fleet_status.via_year == c) & 
                                                (fleet_status.via_month == fleet_status[fleet_status.via_year == c].via_month.max())
                                               ].is_electric.mean()*100 for c in elect.year]))
  elect['actual'] = elect['year'].map(dict_electrification)
  elect['ratio'] = elect['actual']/elect['plan']*100
  
  # Select columns to stick with
  elect_cols = ['plan', 'year', 'actual', 'ratio']
  elect = elect[elect_cols]
  return elect

In [0]:
def electrification_scenarios(clean_viaturas):
    ''' Auxiliary function to generate the electrification scenarios. Given:
      - # of KM done by electric cars
      - % of electric vehicles in the fleet
      - % of yearly fleet increase
    computes the CO2 saved taking as an assumption that on average 50g of CO2 are not emitted in contrast with typical combustão vehicles'''
    
    print('7. Creating electrification scenarios :::', time.ctime())
    
    # Select operative viaturas from the last snapshot
    last_snapshot = clean_viaturas[clean_viaturas.via_status.isin(['IMOB. POR ACIDENTE','OPERACIONAL'])]
    last_snapshot = last_snapshot[last_snapshot['snapshot_date'] == last_snapshot['snapshot_date'].max()]
    current_fleet = last_snapshot.via_plate.nunique()
    
    # Run scenarios with predefined possible inputs for the different variables
    elec_scenarios = pd.DataFrame()
    for year in [2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030]:
        for km_car in [5000,7500,10000,1250,15000,17500,20000,22500,25000]:
            for perc_elec in [5,10,15,20,25,30,35,40,45,50,55,60]:
                for fleet_increase in [-20,-15,-10,-5,0,5,10,15,20]:
                    to_append = pd.DataFrame({'year': [year],
                                              'electric_km': [km_car],
                                              'perc_elect_fleet': [perc_elec],
                                              'perc_inc_fleet': [fleet_increase],
                                              'number_elec_cars': [current_fleet*((1+fleet_increase/100)**(year-2019))*perc_elec/100],
                                              'number_elec_kms': [current_fleet*((1+fleet_increase/100)**(year-2019))*perc_elec/100*km_car],
                                              'co2_saved': [current_fleet*((1+fleet_increase/100)**(year-2019))*perc_elec/100*km_car*0.05]})
                    elec_scenarios = pd.concat([elec_scenarios, to_append])
    
    # Select columns to stick with
    elec_scenarios_cols = ['co2_saved', 'electric_km', 'number_elec_cars', 'number_elec_kms', 'perc_elect_fleet', 'perc_inc_fleet', 'year']
    elec_scenarios = elec_scenarios[elec_scenarios_cols]
    return elec_scenarios

In [0]:
def utilization_table(movimentos, viaturas):
  
  '''Auxiliary table to generate the utilization analysis for the current fleet '''
  
  print('8. Creating Utilization table :::', time.ctime())
  
  clean_movimentos = movimentos.copy()
  clean_movimentos['year_quarter'] = clean_movimentos.mov_movement_date.apply(lambda x: x.year).astype(str)+'_'+clean_movimentos.mov_movement_date.apply(lambda x: x.quarter).astype(str)
  clean_movimentos['date_quarter'] = clean_movimentos['year_quarter'].map(clean_movimentos.groupby('year_quarter').mov_movement_date.min()) + pd.offsets.MonthEnd(3)
  clean_movimentos['key'] = clean_movimentos['date_quarter'].astype(str) + '_' + clean_movimentos['mov_plate']
  
  
  # Subset movimentos to stick with Servicio, Ligeros, Combustion and operational
  clean_movimentos['attribution'] = clean_movimentos['mov_plate'].map(dict(zip(viaturas.via_plate, viaturas.via_attribution)))
  clean_movimentos['category'] = clean_movimentos['mov_plate'].map(dict(zip(viaturas.via_plate, viaturas.via_category)))
  clean_movimentos['fuel'] = clean_movimentos['mov_plate'].map(dict(zip(viaturas.via_plate, viaturas.via_fuel)))
  clean_movimentos = clean_movimentos[(clean_movimentos.attribution == 'Viaturas de Serviço') & 
                                      (clean_movimentos.category == 'LIGEIRO') & 
                                      (clean_movimentos.fuel == 'Combustão') & 
                                      (clean_movimentos.mov_category_of_vehicle.isin(['OPERACIONAL','IMOB. POR ACIDENTE']))]
  
  # Create utilization table based on abastecimentos
  utilization = clean_movimentos[clean_movimentos.mov_movement_type == 'ABASTECIMENTO'].groupby(['date_quarter', 'mov_plate'])['mov_quantity'].sum().reset_index()
  utilization['is_last_period'] = utilization['date_quarter'] == utilization['date_quarter'].max()
  utilization['trimester'] = utilization['date_quarter'].apply(lambda x: add_trimester(x))
  
  # Add predicted KM as a regression of voltage, liters and gross weight corrected for extreme values
  utilization['voltage_liters'] = utilization['mov_plate'].map(dict(zip(viaturas.via_plate, viaturas.via_cc_voltage))) * utilization.mov_quantity
  utilization['gross_weight_liters'] = utilization['mov_plate'].map(dict(zip(viaturas.via_plate, viaturas.via_gross_weight))) * utilization.mov_quantity
  utilization['predicted_kms'] = utilization['mov_quantity'] * 2.14111935e+01 - 2.15275095e-04*utilization['voltage_liters'] - 4.03502335e-03*utilization['gross_weight_liters']
  utilization.loc[(utilization.predicted_kms<=0), 'predicted_kms'] = utilization.mov_quantity * 13
  
  
  # Add utilization KPIs like age, number of repairs and indicators to filter
  utilization['city'] = utilization['mov_plate'].map(dict(zip(viaturas.via_plate, viaturas.via_city)))
  utilization['date_registration'] = utilization['mov_plate'].map(dict(zip(viaturas.via_plate, viaturas.via_date_of_registration)))
  utilization['years_old'] = (utilization['date_quarter'] - pd.to_datetime(utilization.date_registration)).dt.days/365
  utilization['owner_company'] = utilization['mov_plate'].map(dict(zip(movimentos.mov_plate, movimentos.mov_owner_company)))
  utilization['cost_center'] = utilization['mov_plate'].map(dict(zip(movimentos.mov_plate, movimentos.mov_cost_center)))
  utilization['category'] = utilization['mov_plate'].map(dict(zip(viaturas.via_plate, viaturas.via_category)))
  utilization['key'] = utilization['date_quarter'].astype(str)+'_'+utilization['mov_plate']
  utilization['n_repairs'] = utilization['key'].map(clean_movimentos[clean_movimentos.mov_movement_type.isin(['SCHEDULED', 'NO SCHEDULED', 'TIRES'])].groupby('key').size()).fillna(0)
  
  
  # Add bins of the KM and the age for the barchart
  utilization['bins_km'] = utilization.predicted_kms.apply(lambda x: bins_kms3(x))
  utilization['bins_repairs'] = utilization.n_repairs.apply(lambda x: bins_repairs3(x))
  
  # Select columns to stick with
  utilization_cols = ['date_quarter','mov_plate','mov_quantity','is_last_period','trimester','voltage_liters',
                      'gross_weight_liters','predicted_kms','city','date_registration','years_old','owner_company',
                      'cost_center','category','key','n_repairs','bins_km','bins_repairs']
  utilization = utilization[utilization_cols]
  
  return utilization

In [0]:
def replacement_forecast(clean_viaturas, clean_movimentos):
    '''Auxiliary function that creates a forecast of the vehicles that will be obsolete in terms of age / Kms according to company policy in the upcoming years'''
    
    print('9. Creating Replacement forecast table :::', time.ctime())
    
    # Select operative vehicles in the latest snapshot
    replacement_projections = clean_viaturas[clean_viaturas.via_status.isin(['IMOB. POR ACIDENTE','OPERACIONAL'])]
    replacement_projections = replacement_projections[replacement_projections['snapshot_date'] == replacement_projections['snapshot_date'].max()]
    
    # Select some columns and define the KM per year per vehicle
    replacement_projections['company'] = replacement_projections['via_company']
    replacement_projections = replacement_projections[['via_plate', 'via_brand', 'via_attribution', 'via_category',
                                                      'via_city', 'company', 'via_fuel','via_total_kms', 'years_old']]
    replacement_projections['avg_km_per_year'] = replacement_projections['via_total_kms'] / replacement_projections['years_old'] 
    
    # Add months ahead variable to later compute the replacement forecast
    months_ahead = [1,2,3,4,5,6,7,8,9,10,11,12]
    all_replacement_projections = pd.DataFrame()
    for m in months_ahead:
        temp = replacement_projections.copy()
        temp['months_ahead'] = m
        all_replacement_projections = pd.concat([all_replacement_projections, temp])
    all_replacement_projections.reset_index(drop = True, inplace = True)

    # Define the kms and years each given vehicle will have at m months ahead of time
    all_replacement_projections['kms_ahead'] = (all_replacement_projections['via_total_kms']+
                                                all_replacement_projections['months_ahead']*all_replacement_projections['avg_km_per_year']/12)
    all_replacement_projections['years_ahead'] = (all_replacement_projections['years_old']+
                                                all_replacement_projections['months_ahead']/12)
    
    # Default limits for vehicle replacement
    all_replacement_projections['above_km'] = (all_replacement_projections['kms_ahead']>250000).astype(float)
    all_replacement_projections['above_years'] = (all_replacement_projections['years_ahead']>8).astype(float)
    
    # Limits for servicio
    all_replacement_projections.loc[(all_replacement_projections.via_attribution == 'SERVIÇO') & 
                                    (all_replacement_projections.via_fuel.isin(['GASÓLEO', 'GASOLINA'])), 'above_km'] = (all_replacement_projections['kms_ahead']>200000).astype(float)
    all_replacement_projections.loc[(all_replacement_projections.via_attribution == 'SERVIÇO') & 
                                    (all_replacement_projections.via_fuel.isin(['GASÓLEO', 'GASOLINA'])), 'above_years'] = (all_replacement_projections['years_ahead']>8).astype(float)
    
    all_replacement_projections.loc[(all_replacement_projections.via_attribution == 'SERVIÇO') & 
                                    (all_replacement_projections.via_fuel.isin(['ENERGIA ELÉCTRICA'])), 'above_km'] = (all_replacement_projections['kms_ahead']> 160000).astype(float)
    
    # Limits for VUP
    all_replacement_projections.loc[(all_replacement_projections.via_attribution.isin(['VUP', 'VUP/CA'])) & 
                                    (all_replacement_projections.via_fuel.isin(['ENERGIA ELÉCTRICA', 'PLUG-IN GASOLINA'])), 
                                    'above_km'] = (all_replacement_projections['kms_ahead']>150000).astype(float)
    all_replacement_projections.loc[(all_replacement_projections.via_attribution.isin(['VUP', 'VUP/CA'])) & 
                                    (all_replacement_projections.via_fuel.isin(['ENERGIA ELÉCTRICA', 'PLUG-IN GASOLINA'])), 
                                    'above_years'] = (all_replacement_projections['years_ahead']>4).astype(float)
    
    
    all_replacement_projections.loc[(all_replacement_projections.via_attribution.isin(['VUP', 'VUP/CA'])) & 
                                    (all_replacement_projections.via_fuel.isin(['GASÓLEO', 'GASOLINA', 'HÍBRIDO GASOLINA', 'HÍBRIDO GASÓLEO'])), 
                                    'above_km'] = (all_replacement_projections['kms_ahead']>150000).astype(float)
    all_replacement_projections.loc[(all_replacement_projections.via_attribution.isin(['VUP', 'VUP/CA'])) & 
                                    (all_replacement_projections.via_fuel.isin(['GASÓLEO', 'GASOLINA', 'HÍBRIDO GASOLINA', 'HÍBRIDO GASÓLEO'])), 
                                    'above_years'] = (all_replacement_projections['years_ahead']>5).astype(float)
    
    # Select columns to stick with
    replacement_cols = ['via_plate','via_brand','via_attribution','via_category','via_city','company','via_fuel','via_total_kms','years_old',
                        'avg_km_per_year', 'months_ahead','kms_ahead','years_ahead','above_km','above_years']
    all_replacement_projections = all_replacement_projections[replacement_cols]

    return all_replacement_projections

In [0]:
def financial_scenarios(clean_viaturas, clean_movimentos):
  
    print('10. Creating financial scenarios :::', time.ctime())
    
    fleet_status = clean_viaturas[clean_viaturas.via_status.isin(['IMOB. POR ACIDENTE','OPERACIONAL'])]
    fleet_status = fleet_status[fleet_status['snapshot_date'] == fleet_status['snapshot_date'].max()]
    current_operative = fleet_status.copy()

    clean_movimentos = clean_movimentos[clean_movimentos.mov_plate.isin(current_operative.via_plate)]
    clean_movimentos = clean_movimentos[clean_movimentos.mov_movement_date >= pd.Timestamp.today() - pd.Timedelta('365 days')]

    current_operative['min_kms'] = current_operative.via_plate.map(
        clean_movimentos[clean_movimentos.mov_kms.fillna(0) != 0].groupby('mov_plate').mov_kms.min())
    current_operative['max_kms'] = current_operative.via_plate.map(
        clean_movimentos[clean_movimentos.mov_kms.fillna(0) != 0].groupby('mov_plate').mov_kms.max())
    current_operative.loc[current_operative.max_kms > 500000, 'max_kms'] = 100000
    current_operative['done_kms'] = current_operative['max_kms'] - current_operative['min_kms']
    current_operative = current_operative[current_operative.done_kms.notnull()]
    current_operative['is_electric'] = current_operative.via_fuel.isin(['ENERGIA ELÉCTRICA', 'PLUG-IN GASOLINA'])

    # We define the fixed data for the scenarios
    exp_scenarios = (fleet_status.groupby('via_company').size()).reset_index()
    exp_scenarios.columns = ['via_company', 'number_cars']
    exp_scenarios['share_in_fleet'] = exp_scenarios['number_cars'] / exp_scenarios['number_cars'].sum()
    exp_scenarios['km_done'] = exp_scenarios.company.map(current_operative.groupby('via_company')['done_kms'].sum())
    exp_scenarios['perc_electric'] = exp_scenarios['company'].map(current_operative.groupby('via_company')['is_electric'].mean())

    # We append data by expense type
    res = pd.DataFrame()
    for t in clean_movimentos.mov_movement_type.unique():
        temp = exp_scenarios.copy()
        temp['expense_type'] = t
        temp['eur_km'] = exp_scenarios.company.map(
            clean_movimentos[clean_movimentos.mov_movement_type == t].groupby(
                'mov_owner_company').mov_total_net_price.sum()).fillna(0) / exp_scenarios['km_done']
        res = pd.concat([res, temp])
    res = res.fillna(0).replace([np.inf,-np.inf],0).reset_index(drop = True)

    # We generate the scenarios according to each and every input
    cost_scenario = pd.DataFrame()
    for serv_increase in [-10,-5,-2,0,2,5,10]:
        for cars_dismissed in [200-c*40 for c in range(6)]:
            for cars_purchased in [400-c*40 for c in range(11)]:
                for perc_elect in [0,5,10,15,20,25,30]:
                    for price_new_cars in [10000,11000,12000,13000,14000,15000]:
                        temp = res.copy()
                        temp['serv_increase'] = serv_increase
                        temp['cars_dismissed'] = cars_dismissed
                        temp['cars_purchased'] = cars_purchased
                        temp['perc_elect'] = perc_elect
                        temp['price_new_cars'] = price_new_cars
                        temp['scenario_ncars'] = temp['number_cars'] + cars_purchased - cars_dismissed
                        temp['scenario_km_per_car'] = temp['km_done'] * (serv_increase/100+1) / temp.scenario_ncars
                        temp['scenario_eur'] = temp['eur_km']*temp['scenario_ncars']*temp['scenario_km_per_car']
                        temp.loc[temp.expense_type == 'ABASTECIMENTO','scenario_eur'] = temp['scenario_eur']-temp['scenario_eur']*perc_elect/100/2
                        temp['capex_expense'] = cars_purchased * price_new_cars
                        temp['capex_income'] = cars_dismissed * 3000
                        temp['capex_balance'] = temp['capex_expense'] - temp['capex_income']
                        temp['delta_capex'] = (cars_dismissed - cars_purchased) * price_new_cars
                        cost_scenario = pd.concat([cost_scenario, temp])
    cost_scenario.reset_index(drop = True, inplace = True)
    
    return cost_scenario

In [0]:
def matricula_table(viaturas, clean_movimentos):
  
  print('11. Creating Matrícula details table :::', time.ctime())
  
  clean_viaturas = viaturas.sort_values('snapshot_date', ascending = False).drop_duplicates('via_plate')
  matricula = pd.DataFrame({'via_plate': clean_viaturas.via_plate.unique()})
  
  # Static vehicle information
  matricula['brand'] = matricula.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_brand)))
  matricula['model'] = matricula.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_model)))
  matricula['category'] = matricula.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_category)))
  matricula['classification'] = matricula.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_classification)))
  matricula['attribution'] = matricula.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_attribution)))
  matricula['via_co2_emissions'] = matricula.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_co2_emissions)))
  matricula['fuel'] = matricula.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_fuel)))
  matricula['city'] = matricula.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_city)))
  matricula['date_of_registration'] = pd.to_datetime(matricula.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_date_of_registration))))
  matricula['acquisition_value'] = matricula.via_plate.map(clean_viaturas.groupby('via_plate').via_acquisition_value.max())
  matricula['cost_center'] = matricula.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_cost_center)))
  matricula['total_kms'] = matricula.via_plate.map(clean_viaturas.groupby('via_plate').via_total_kms.max())
  matricula['owner_company'] = matricula.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_company)))
  matricula['cc_voltage'] = matricula.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_cc_voltage)))
  matricula['gross_weight'] = matricula.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_gross_weight)))
  matricula['status'] = matricula.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.via_status)))
  matricula['years_old'] = matricula.via_plate.map(dict(zip(clean_viaturas.via_plate, clean_viaturas.years_old)))
  
  # Usage stats
  mov_cols = ['mov_plate', 'mov_movement_date', 'mov_total_net_price', 'mov_quantity', 'mov_movement_type']
  matricula = pd.merge(matricula, 
                       clean_movimentos[mov_cols],
                       left_on='via_plate',
                       right_on='mov_plate',
                       how='left')
  matricula['trimester'] = matricula['mov_movement_date'].apply(lambda x: add_trimester(x))
  
  # Computing km driven using liters
  mask = (matricula.mov_movement_type == "ABASTECIMENTO")
  matricula['km_driven'] = 0
  matricula.loc[mask, 'km_driven'] =  (2.14111935e+01* matricula['mov_quantity'] - 
                                      2.15275095e-04* matricula['mov_quantity'] * matricula['cc_voltage'] -
                                      4.03502335e-03 *matricula['mov_quantity'] * matricula['gross_weight'])
  matricula['category_classification'] = matricula['category'] + '_' + matricula['classification']
  
  return matricula

In [0]:
def tables_excel(viaturas, movimentos):
  print('12. Creating excel reports table :::', time.ctime())
  currentviaturas = viaturas[viaturas.via_status.isin(['IMOB. POR ACIDENTE','OPERACIONAL'])]
  currentviaturas['snapshot_date'] = pd.to_datetime(currentviaturas['via_year'].astype(str)+'-'+currentviaturas['via_month'].astype(str))
  currentviaturas = currentviaturas[currentviaturas['snapshot_date'] == currentviaturas['snapshot_date'].max()]

  currentviaturas['eur_spent'] = currentviaturas.via_plate.map(movimentos.groupby('mov_plate')['mov_total_net_price'].sum())

  table1 = currentviaturas.groupby(['via_attribution', 'via_category']).eur_spent.agg(Veículos= 'count',Euros='sum').reset_index()
  table1['Custo Unitário'] = table1['Euros'] / table1['Veículos']
  
  ###########################################################################
  
  def km_bins(x):
    if (x == 0): return '1. 0 Kms'
    if (x < 4500): return '2. 1-4500 Km'
    if (x < 9000): return '3. 4501-9000 Km'
    if (x < 13500): return '4. 9001-13500 Km'
    if (x < 18000): return '5. 1501-18000 Km'
    if (x >= 18000): return '6. >18000 Km'
    return 'not'
  table2 = currentviaturas['via_total_kms'].apply(lambda x: km_bins(x)).value_counts().reset_index().sort_values('index')
  table2.columns = ['Km Percorridos', '#']
  
  ###########################################################################
  
  movimentos['is_last_period'] = (pd.to_datetime(movimentos['mov_movement_date']) >= pd.to_datetime(str(movimentos['mov_movement_date'].max().year)+'-01')).astype(float)
  my_movimentos = movimentos[(movimentos.mov_plate.isin(currentviaturas.via_plate)) & (movimentos.is_last_period == True)]
  currentviaturas['consumos'] = currentviaturas.via_plate.map(my_movimentos[my_movimentos.mov_movement_type == 'ABASTECIMENTO'].groupby('mov_plate').mov_quantity.sum()).fillna(0)
  currentviaturas['absatecimento'] = currentviaturas.via_plate.map(my_movimentos[my_movimentos.mov_movement_type == 'ABASTECIMENTO'].groupby('mov_plate').mov_total_net_price.sum()).fillna(0)

  my_movimentos['liters_cc'] = 2#my_movimentos['mov_quantity'] * my_movimentos.mov_plate.map(dict(zip(currentviaturas.via_plate, currentviaturas.via_cc_voltage)))
  my_movimentos['liters_weight'] = 2#my_movimentos['mov_quantity'] * my_movimentos.mov_plate.map(dict(zip(currentviaturas.via_plate, currentviaturas.via_gross_weight)))


  my_movimentos.loc[my_movimentos.mov_movement_type == 'ABASTECIMENTO', 'predicted_kms'] =  2#(19.4091301*my_movimentos.mov_quantity -
                                                                                            # 2.37490333e-03* my_movimentos.liters_cc -
                                                                                            # 2.08773361e-03*my_movimentos.liters_weight)
  my_movimentos.predicted_kms.fillna(0, inplace = True)
  currentviaturas['predicted_kms'] = currentviaturas.via_plate.map(my_movimentos[my_movimentos.mov_movement_type == 'ABASTECIMENTO'].groupby('mov_plate').predicted_kms.sum()).fillna(0)
  
  
  
  table3 = currentviaturas.groupby('via_fuel').size().reset_index()
  table3.columns = ['Tipo de Combustível', 'Veículos']
  table3['% Combustível'] = round(table3['Veículos'] / table3['Veículos'].sum()*100,2)
  table3['KM'] = table3['Tipo de Combustível'].map(currentviaturas.groupby('via_fuel').predicted_kms.sum())
  table3['Consumos'] = table3['Tipo de Combustível'].map(currentviaturas.groupby('via_fuel')['consumos'].sum())
  table3['Abastecimentos'] = table3['Tipo de Combustível'].map(currentviaturas.groupby('via_fuel')['absatecimento'].sum())
  table3['€/Consumo'] = table3['Abastecimentos'] / table3['Consumos']
  table3['€/KM'] = table3['Abastecimentos'] / table3['KM']
  
  
  ###########################################################################
  
  table4 = movimentos.copy()
  table4['mov_movement_date'] = pd.to_datetime(table4['mov_movement_date'])
  table4['is_last_period'] = (table4['mov_movement_date'] >= pd.to_datetime(str(table4['mov_movement_date'].max().year)+'-01')).astype(float)
  table4 = table4[table4.is_last_period == True]
  table4 = table4.groupby('mov_movement_type')['mov_total_net_price'].sum().reset_index()
  table4.columns = ['Categoria custo', 'Valor']
  table4['%'] = table4['Valor'] / table4['Valor'].sum()
  
  ###########################################################################
  
  
  
  return table1, table2, table3, table4


In [0]:
viaturas = load_viaturas()

In [0]:
viaturas = load_viaturas()
movimentos = load_movimentos()
balances = fleet_balances_table(viaturas, movimentos)
capex = capex_table(balances, viaturas, movimentos)

In [0]:
save_in_blob(capex, '/out/csv/03_capex_v7')

In [0]:
def main():
  '''Function that triggers all of the previously displayed functionalities'''
  viaturas = load_viaturas()
  movimentos = load_movimentos()
  cadastro = load_cadastro()
  
  filtering_slicer = filters_table(viaturas)
  save_in_blob(filtering_slicer, '/out/csv/00_Filters_v10')
  
  fleet_status = fleet_status_table(movimentos, viaturas, cadastro)
  save_in_blob(fleet_status, '/out/csv/01_Fleet_Status_v9')
  
  balances = fleet_balances_table(viaturas, movimentos)
  save_in_blob(balances, '/out/csv/02_Fleet_Balances_v7')
  
  capex = capex_table(balances, viaturas, movimentos)
  save_in_blob(capex, '/out/csv/03_capex_v7')
  
  opex_breakdown = opex_table(viaturas, movimentos)
  save_in_blob(opex_breakdown, '/out/csv/04_Opex_v6')
  
  portagens = portagens_times_table(movimentos)
  save_in_blob(portagens, '/out/csv/04_portagenstimes_v3')
  
  cost_calculator = compute_costcalculator_table(viaturas, movimentos)
  save_in_blob( cost_calculator, '/out/csv/05_fatigue_v7')
  
  electrification = electrification_table(viaturas)
  save_in_blob(electrification, '/out/csv/06_electrification_v2')
  
  electric_scenarios = electrification_scenarios(viaturas)
  save_in_blob(electric_scenarios, '/out/csv/07_elec_scenarios_v1')
  
  utilization = utilization_table(movimentos, viaturas)
  save_in_blob(utilization, '/out/csv/08_utilization_v9')
  
  car_replacements = replacement_forecast(viaturas, movimentos)
  save_in_blob(car_replacements, '/out/csv/09_car_replacements_v6')
  
  #costs = financial_scenarios(viaturas, movimentos)
  #save_in_blob(costs, '10_costs_v1')
  
  matricula = matricula_table(viaturas, movimentos)
  save_in_blob(matricula, '/out/csv/11_matricula_details_v6')
  
  table1, table2, table3, table4 = tables_excel(viaturas, movimentos)
  save_in_blob(table1, '/out/csv/12_exceltables1_v1')
  save_in_blob(table2, '/out/csv/12_exceltables2_v1')
  save_in_blob(table3, '/out/csv/12_exceltables3_v1')
  save_in_blob(table4, '/out/csv/12_exceltables4_v1')
  
  return 'DONE!'

  

In [0]:
main()